<a href="https://colab.research.google.com/github/viczzzz/Capstone-Fashion-Trend-Forecasting/blob/main/DeepFashion_Milestone1_Detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Wed Apr  6 17:45:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()
# Standard output is '/device:GPU:0'

'/device:GPU:0'

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! ls 

'Colab Notebooks'   Images


In [7]:
import os 
os.getcwd()
#os.chdir('/content/drive/MyDrive')
os.chdir('/content/drive/MyDrive')

In [ ]:
#!fusermount -u drive
#!google-drive-ocamlfuse drive

In [9]:
!apt install -y caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-numpy python3-olefile
  python3-pandas python3-pandas-lib python3-pexpect python3-pickleshare
  python3-pil python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  python3-scipy python3-simplegeneric python3-six python3-skimage
  p

In [13]:
# COCOAPI=/path/to/clone/cocoapi
! git clone https://github.com/cocodataset/cocoapi.git $COCOAPI
#cd $COCOAPI/PythonAPI
# Install into global site-packages
# Alternatively, if you do not have permissions or prefer
# not to install the COCO API into global site-packages
#python setup.py install --user

SyntaxError: ignored

In [15]:
! git clone https://github.com/facebookresearch/Detectron.git


Cloning into 'Detectron'...
remote: Enumerating objects: 1377, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1377 (delta 1), reused 2 (delta 0), pack-reused 1370
Receiving objects: 100% (1377/1377), 4.38 MiB | 16.63 MiB/s, done.
Resolving deltas: 100% (825/825), done.


In [22]:
os.chdir('Detectron')
!pip install -r requirements.txt


     |████████████████████████████████| 375 kB 10.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-3.12-cp37-cp37m-linux_x86_64.whl size=43073 sha256=3ec7c5def23eb52767292a394119f37cca71a353b25428cc80c97cdef8a36aef
  Stored in directory: /root/.cache/pip/wheels/21/cc/0a/8f691c938b079b07fa17a2e2940e3faae0a3dfb3ff5b3ac356
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
! ls

cmake		    demo		INSTALL.md    projects
CMakeLists.txt	    detectron		LICENSE       README.md
CODE_OF_CONDUCT.md  docker		Makefile      requirements.txt
configs		    FAQ.md		MODEL_ZOO.md  setup.py
CONTRIBUTING.md     GETTING_STARTED.md	NOTICE	      tools


In [24]:
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
from random import shuffle
import os

categories_dict ={1: 'upper', 2: 'upper', 3: 'upper', 4: 'upper', 5: 'upper',
                  6: 'upper', 7: 'upper', 8: 'upper', 9: 'upper', 10: 'upper',
                  11: 'upper', 12: 'upper', 13: 'upper', 14: 'upper', 15: 'upper',
                  16: 'upper', 17: 'upper', 18: 'upper', 19: 'upper', 20: 'upper',
                  21: 'lower', 22: 'lower', 23: 'lower', 24: 'lower', 25: 'lower',
                  26: 'lower', 27: 'lower', 28: 'lower', 29: 'lower', 30: 'lower',
                  31: 'lower', 32: 'lower', 33: 'lower', 34: 'lower', 35: 'lower',
                  36: 'lower',
                  37: 'full', 38: 'full', 39: 'full', 40: 'full', 41: 'full',
                  42: 'full', 43: 'full', 44: 'full', 45: 'full', 46: 'full',
                  47: 'full', 48: 'full', 49: 'full', 50: 'full' }
categories3_dict = {'upper': 1, 'lower': 2, 'full': 3}


def to_coco(shuffle_idx, images, bboxs, imagesdir):

    coco_dict = {}
    coco_dict[u'images'] = []
    coco_dict[u'annotations'] = []
    count = 0
    for idx in shuffle_idx:
        print ('---', count, '---', len(shuffle_idx))
        count += 1
        assert bboxs[idx].split(' ')[0] == images[idx].split(' ')[0]

        image_name = images[idx].split(' ')[0]
        category_id = eval(images[idx].split(' ')[-1])
        image_file = imagesdir + 'Img/' + image_name
        img = cv2.imread(image_file, 0)
        height, width = img.shape

        x1, y1, x2, y2 = map(int, bboxs[idx].split(' ')[-4:])
        # img = cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # plt.imshow(img)
        # plt.show()

        ###
        image_dict = {u'date_captured': u'2019-05-03 14:01:02',
                      u'file_name': image_name,
                      u'height': height,
                      u'id': idx,
                      u'license': 1,
                      u'url': image_name,
                      u'width': width}
        coco_dict[u'images'].append(image_dict)

        ###
        bbox = [x1, y1, x2 - x1, y2 - y1]  # [x,y,width,height]
        annotation_dict = {u'segmentation': [[312.29, 562.89]],
                           u'area': (x2 - x1) * (y2 - y1),
                           u'iscrowd': 0, # 
                           u'image_id': idx,
                           u'bbox': bbox,
                           u'category_id': categories3_dict[categories_dict[category_id]],
                           u'id': idx}
        coco_dict[u'annotations'].append(annotation_dict)

    ###
    coco_info_dict = {u'contributor': u'DeepFashion',
                      u'date_created': u'2019-05-03 14:01:02',
                      u'description': u'COCO Dataset format of DeepFashion.',
                      u'url': u'',
                      u'version': u'',
                      u'year': 2019}
    coco_type = u'instances'
    coco_licenses_dict = [{u'id': 1,
                           u'name': u'Attribution-NonCommercial-ShareAlike License',
                           u'url': u'http://creativecommons.org/licenses/by-nc-sa/2.0/'}]
    coco_categories = [{u'id': 1, u'name': u'upper', u'supercategory': u'upper'},
                       {u'id': 2, u'name': u'lower', u'supercategory': u'lower'},
                       {u'id': 3, u'name': u'full', u'supercategory': u'full'}]


    coco_dict[u'info'] = coco_info_dict
    coco_dict[u'licenses'] = coco_licenses_dict
    coco_dict[u'categories'] = coco_categories

    return coco_dict


if __name__ == '__main__':
    print ('Convert DeepFashion Data to COCO Format...')

    deepfashion_root = '/home/ubuntu/DeepFashion/Category and Attribute Prediction Benchmark/'

    category_annofile = deepfashion_root + 'Anno/list_category_cloth.txt'
    category_imgfile = deepfashion_root + 'Anno/list_category_img.txt'
    category_bboxfile = deepfashion_root + 'Anno/list_bbox.txt'

    categories = open(category_annofile).readlines()[2:]
    images = open(category_imgfile).readlines()[2:]
    bboxs = open(category_bboxfile).readlines()[2:]

    assert len(images) == len(bboxs)
    print ('Num of Deepfashion Category Images: ', len(images))

    shuffle_idx = list(range(len(images)))
    shuffle(shuffle_idx)

    coco_dict_train = to_coco(shuffle_idx[:200000], images, bboxs, deepfashion_root)
    coco_dict_val = to_coco(shuffle_idx[200000:], images, bboxs, deepfashion_root)

    f = open('coco_deepfashion3_train.json', 'w')
    f.write(json.dumps(coco_dict_train))
    f.close()

    f = open('coco_deepfashion3_val.json', 'w')
    f.write(json.dumps(coco_dict_val))
    f.close()

    print('Done.')

Convert DeepFashion Data to COCO Format...


FileNotFoundError: ignored